In [ ]:
!pip install fastapi easyocr pyngrok uvicorn python-multipart
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import easyocr
import io
from PIL import Image
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import pyngrok.ngrok
import asyncio
import signal
from pydantic import BaseModel
import base64
from typing import List, Tuple, Dict, Any
import numpy as np

app = FastAPI(title="EasyOCR API")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

def convert_bbox_to_serializable(bbox) -> List[List[float]]:
    return [[float(x) for x in point] for point in bbox]

reader = easyocr.Reader(['en', 'ru'])


@app.post("/ocr")
async def recognize_text(file: UploadFile = File(...)):
    try:
        if not file.content_type.startswith('image/'):
            raise HTTPException(status_code=400, detail="Файл должен быть изображением")
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image_np = np.array(image)
        result = reader.readtext(image_np)
        texts = [detection[1] for detection in result]
        full_text = " ".join(texts)

        serializable_result = []
        for detection in result:
            bbox = []
            for point in detection[0]:
                if hasattr(point, 'tolist'):
                    bbox.append(point.tolist())
                else:
                    bbox.append([float(x) for x in point])

            # Преобразуем confidence score
            confidence = float(detection[2])

            serializable_result.append({
                "bbox": bbox,
                "text": detection[1],
                "confidence": confidence
            })

        return {
            "status": "success",
            "text": full_text,
            "details": serializable_result
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Ошибка обработки: {str(e)}")


class OcrItem(BaseModel):
    image_base64: str

@app.post("/ocr-base")
async def recognize_text(
    item: OcrItem
):
    try:
        if "," in item.image_base64:
            item.image_base64 = item.image_base64.split(",")[1]

        image_data = base64.b64decode(item.image_base64)
        image = Image.open(io.BytesIO(image_data))

        result = reader.readtext(image)
        print('result', result)
        processed_result = []
        for detection in result:
            bbox, text, confidence = detection
            serializable_bbox = convert_bbox_to_serializable(bbox)
            serializable_confidence = float(confidence)

            processed_result.append(
                (serializable_bbox, text, serializable_confidence)
            )

        texts = [detection[1] for detection in processed_result]
        full_text = " ".join(texts)

        return {
            "status": "success",
            "text": full_text,
            "details": processed_result,
        }

    except Exception as e:
        raise HTTPException(
            status_code=400,
            detail=f"Ошибка обработки изображения: {str(e)}"
        )

async def shutdown_signal_handler():
    print("Shutting down server...")

def main():
    NGROK_AUTH_TOKEN = ""  # токен свой бери
    pyngrok.ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    ngrok_tunnel = pyngrok.ngrok.connect(8000)
    print('👉 Твой URL API:', ngrok_tunnel.public_url)

    config = uvicorn.Config(
        app,
        host="0.0.0.0",
        port=8000,
        log_level="debug"
    )
    server = uvicorn.Server(config)

    loop = asyncio.get_event_loop()
    for sig in (signal.SIGTERM, signal.SIGINT):
        loop.add_signal_handler(
            sig,
            lambda: asyncio.create_task(shutdown_signal_handler())
        )

    nest_asyncio.apply()
    try:
        loop.run_until_complete(server.serve())
    except KeyboardInterrupt:
        print("Server stopped by user")
    finally:
        pyngrok.ngrok.disconnect(ngrok_tunnel.public_url)
        print("ngrok tunnel closed")


main()